In [ ]:
import openreview
from tqdm import tqdm
from collections import defaultdict
from openreview.api import OpenReviewClient

In [ ]:
openreview_client = OpenReviewClient(baseurl='https://api2.openreview.net',username='',password='')

In [ ]:
venue_id = '' # use the venueID for the current cycle making the request

In [ ]:
reviewers_group = openreview_client.get_group(f'{venue_id}/Reviewers').members
print(len(reviewers_group))

acs_group = openreview_client.get_group(f'{venue_id}/Area_Chairs').members
print(len(acs_group))

In [ ]:
# get resubmissions
submissions = openreview_client.get_all_notes(content={"venueid": f"{venue_id}/Submission"})
resubmissions = []
no_resubmissions = []
for submission in submissions:
    
    try:
        if 'Yes' in submission.content['resubmission_flag']['value']:
            resubmissions.append(submission)
    except KeyError:
        no_resubmissions.append(submission)

len(resubmissions)

In [ ]:
print(f'Found {len(resubmissions)} resubmissions and {len(no_resubmissions)} non-resubmissions')

In [ ]:
for nr in no_resubmissions:
    print(nr.id)

In [ ]:
print(resubmissions[0].id)

In [ ]:
previous_venue_id = '' # use the venueID of the previous cycle the new cycle wants to check

resubmit_edges = []
for submission in resubmissions:

    if 'paper_link' in submission.content and 'value' in submission.content['paper_link']:
        previous_id = submission.content['paper_link']['value'].split('?id=')[1].split('&')[0].split('#')[0]
        try:
            previous_note = openreview_client.get_note(previous_id)
            if f'{previous_venue_id}/-/Submission' not in previous_note.invitations:
                print(f'Previous note {previous_id} is not a valid submission for {submission.id}')
                previous_note = None
        except Exception as e:
            previous_note = None
            print(f'There was an error with note: {previous_id}: {e}')

        if previous_note:
            previous_revs = openreview_client.get_group(f'{previous_venue_id}/Submission{previous_note.number}/Reviewers')
            previous_acs = openreview_client.get_group(f'{previous_venue_id}/Submission{previous_note.number}/Area_Chairs')
            
            for rev in previous_revs.members:
                if rev in reviewers_group:
                    resubmit_edges.append(openreview.api.Edge(
                        invitation=f'{venue_id}/Reviewers/-/Resubmit_Score',
                        head=submission.id,
                        tail=rev,
                        weight=3,
                        signatures=[f'{venue_id}/Program_Chairs']
                    ))

            for ac in previous_acs.members:
                if ac in acs_group:
                    resubmit_edges.append(openreview.api.Edge(
                        invitation=f'{venue_id}/Area_Chairs/-/Resubmit_Score',
                        head=submission.id,
                        tail=ac,
                        weight=3,
                        signatures=[f'{venue_id}/Program_Chairs']
                    ))
    else:
        print(f"Submission {submission.id} does not have a 'paper_link' field.")

In [ ]:
len(resubmit_edges)

In [ ]:
for edge in resubmit_edges:
    openreview_client.post_edge(edge)

In [ ]:
# uncomment below if you need to delete edges
# openreview_client.delete_edges(invitation = '', wait_to_finish=True)
